In [1]:
import os
import numpy as np
import pandas as pd
import cvxpy as cp
import cplex
import osqp

In [2]:
def read_data_mat100(dataset="tr0"):
    folder = "kernel-methods-for-machine-learning-2018-2019"
    features_file = "X" + dataset + "_mat100.csv"
    labels_file = "Y" + dataset + ".csv"
    
    X = pd.read_csv(
        os.path.join(folder, features_file),
        sep=" ",
        header=None
    )
    if "te" in dataset:
        return np.array(X)
    
    elif "tr" in dataset:
        Y = pd.read_csv(
            os.path.join(folder, labels_file),
            sep=",",
            index_col=0,
        )
        return np.array(X), 2 * np.array(Y.iloc[:, 0]) - 1

In [3]:
def kernel(x1, x2):
    return x1.dot(x2)

In [4]:
X,Y = read_data_mat100("tr0")
m = X.mean(axis=0)
s = X.std(axis=0)
Xc = (X - m)/s

n = len(X)
lambd = 1

I = np.eye(n)
U = (1/n) * np.ones((n, n))
gram_matrix = kernel(Xc, Xc.T)
K = gram_matrix + 1e-9*I

Primal formulation

In [5]:
alpha = cp.Variable(n)
ksi = cp.Variable(n)

constraints1 = [
    ksi >= 0,
    ksi >= 1 - cp.multiply(Y, (K * alpha))
]

objective1 = cp.Minimize(
    (1/n) * cp.sum(ksi)
    + lambd * cp.quad_form(alpha, K)
)

prob1 = cp.Problem(objective1, constraints1)

# result1 = prob1.solve(solver=cp.OSQP, verbose=True)

Dual formulation 1

In [6]:
mu = cp.Variable(n)

constraints2 = [
    mu >= np.zeros(n),
    mu <= np.ones(n) / n
]

objective2 = cp.Minimize(
    - cp.sum(mu)
    + (0.25 / lambd) * cp.quad_form(cp.multiply(mu,Y), K)
)

prob2 = cp.Problem(objective2, constraints2)

# result2 = prob2.solve(solver=cp.OSQP, verbose=True)

Dual formulation 2

In [ ]:
alpha = cp.Variable(n)

constraints3 = [
    cp.multiply(Y, alpha) >= np.zeros(n),
    cp.multiply(Y, alpha) <= np.ones(n) / (2 * lambd * n)
]

objective3 = cp.Minimize(
    - 2 * lambd * (Y * alpha)
    + lambd * cp.quad_form(alpha, K)
)

prob3 = cp.Problem(objective3, constraints3)

result3 = prob3.solve(solver=cp.OSQP, verbose=True)

In [ ]:
def compute_predictor(Xtr, Ytr, kernel, lambd):
    m = X.mean(axis=0)
    s = X.std(axis=0)
    Xc = (X - m)/s

    n = len(X)

    I = np.eye(n)
    U = (1/n) * np.ones((n, n))
    gram_matrix = kernel(Xc, Xc.T)
    K = gram_matrix + 1e-9*I
    
    alpha = cp.Variable(n)

    constraints = [
        cp.multiply(y, alpha) >= np.zeros(n),
        cp.multiply(y, alpha) <= np.ones(n) / (2 * lambd * n)
    ]

    objective = cp.Minimize(
        - 2 * lambd * (Y * alpha)
        + lambd * cp.quad_form(alpha, K)
    )

    prob = cp.Problem(objective, constraints)

    result = prob.solve(solver=cp.OSQP, verbose=True)
    
    return lambda x_new: np.sign(alpha.dot(kernel(Xc, (x_new - m)/s)))